## Imports

In [399]:
import pandas as pd
from datetime import datetime
import re

### JSON

#### ATP data

In [400]:
# JSON loading
with open('data/ATP.json') as file:
    data = json.load(file)

# Access data through key 'us_open_2023'
matches = data['us_open_2023']

# Convert to Dataframe
atp = pd.DataFrame(matches)

#### World Cup

In [401]:
# JSON loading
with open('data/World_Cup_2022.json') as file:
    data = json.load(file)

# Access data through key 'us_open_2023'
matches = data['matches']

# Convert to Dataframe
wc22 = pd.DataFrame(matches)

### Excel 

#### Sports Events

In [402]:
sports = pd.read_excel('data/Sports_events.xlsx')

### CSV

#### IMDB Audiences

In [403]:
imdb = pd.read_csv('data/imdb_data.csv')

#### Movies

In [404]:
movies = pd.read_csv('data/Movies_dataset.csv')

#### Korean Movies

In [405]:
k_movies = pd.read_csv('data/top100_kdrama.csv')

## Data Cleansing

### ATP

In [406]:
atp.dtypes

date            object
match           object
location        object
time            object
final_score     object
rating         float64
reach            int64
dtype: object

We need to change the 2 columns : date and time to put it as date.

#### Date into datetime

In [407]:
atp['date'] = pd.to_datetime(atp['date'])

#### Time into time format

In [408]:
atp['time'] = pd.to_datetime(atp['time'], format='%H:%M').dt.time

#### New Columns Names

In [409]:
atp.columns = ['Date', 'Game', 'Location', 'Time', 'Score', 'Rating', 'Reach']

### World Cup

In [410]:
wc22.dtypes

date              object
match             object
location          object
time              object
final_score       object
viewer_rating    float64
reach_volume       int64
dtype: object

We need to change the 2 columns : date and time to put it as date.

#### Date into date

In [411]:
wc22['date'] = pd.to_datetime(wc22['date'])

#### Time into time format

In [412]:
wc22['time'] = pd.to_datetime(wc22['time'], format='%H:%M').dt.time

#### New Columns Names

In [413]:
wc22.columns = ['Date', 'Game', 'Location', 'Time', 'Score', 'Rating', 'Reach']

### Sports Events

In [414]:
sports.dtypes

Event                object
Date                 object
Location             object
Reach (millions)    float64
Time                 object
Rating               object
dtype: object

We need to change 3 columns : date, time and rating.

#### Date into date

In [415]:
sports['Date'] = pd.to_datetime(sports['Date'], errors='coerce')

/var/folders/5q/f280wjd15s56_y9s5ntbx3rw0000gn/T/ipykernel_98386/2895554504.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  sports['Date'] = pd.to_datetime(sports['Date'], errors='coerce')


#### Time into time format

In [416]:
sports['Time'] = pd.to_datetime(sports['Time'], format='%H:%M', errors='coerce').dt.time

#### Rating into numeric

In [417]:
sports['Rating'] = pd.to_numeric(sports['Rating'], errors='coerce')

sports['Rating'] = sports['Rating'].astype('float64')

In [418]:
sports.head()

,Event,Date,Location,Reach (millions),Time,Rating
0,Event 1,NaT,Arena B,167.62,05:26:00,2.1
1,Event 2,2021-12-03,Ground D,NaN,10:56:00,4.5
2,Event 3,2020-02-03,Arena B,382.39,NaT,4.3
3,Event 4,2019-01-19,Stadium A,377.11,10:19:00,NaN
4,Event 5,2023-12-13,Field C,132.01,01:25:00,9.6


#### Convert Reach into millions

In [419]:
sports['Reach (millions)'] = sports['Reach (millions)']*1000000

In [420]:
sports.rename(columns={'Reach (millions)':'Reach'},inplace=True)

### IMDB Audiences

#### Split Name into 2 columns

In [421]:
# Remplacer les valeurs sans délimiteur par un format uniforme
imdb['Name'] = imdb['Name'].apply(lambda x: x if '\n' in x else x + '\nUnknown')

# Séparer la colonne en deux parties
split_data = imdb['Name'].str.split('\n', expand=True)

# Assigner les colonnes après le split
imdb[['Title', 'Year']] = split_data[[0, 1]]

In [422]:
# Nettoyer la colonne 'Year' pour ne conserver que les chiffres
imdb['Year'] = imdb['Year'].str.extract('(\d+)', expand=False)

# Convertir en entier (optionnel, si vous voulez avoir des valeurs numériques)
imdb['Year'] = pd.to_numeric(imdb['Year'], errors='coerce')

In [423]:
imdb['Year'].fillna(0,inplace=True)

imdb['Year'] = imdb['Year'].astype('int64')

/var/folders/5q/f280wjd15s56_y9s5ntbx3rw0000gn/T/ipykernel_98386/3755493771.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  imdb['Year'].fillna(0,inplace=True)


#### Remove the Name column

In [424]:
imdb.drop(columns=['Name'],inplace=True)

### Movies

#### Remove the index column, which seems useless

In [425]:
movies.drop(columns=['index'],inplace=True)

In [426]:
movies.dtypes

title                 object
original_language     object
release_date          object
popularity           float64
vote_average         float64
vote_count             int64
overview              object
dtype: object

#### Convert the original_language column into real language

In [427]:
# Mapping table to map every alias with a language
language_mapping = {
    'en': 'English',
    'pl': 'Polish',
    'ja': 'Japanese',
    'ar': 'Arabic',
    'fr': 'French',
    'uk': 'Ukrainian',
    'es': 'Spanish',
    'fi': 'Finnish',
    'nl': 'Dutch',
    'ko': 'Korean',
    'zh': 'Chinese',
    'cn': 'Chinese',
    'de': 'German',
    'it': 'Italian',
    'tl': 'Tagalog',
    'ru': 'Russian',
    'no': 'Norwegian',
    'hi': 'Hindi',
    'tr': 'Turkish',
    'te': 'Telugu',
    'pt': 'Portuguese',
    'th': 'Thai',
    'id': 'Indonesian',
    'sv': 'Swedish',
    'da': 'Danish',
    'ga': 'Irish',
    'ta': 'Tamil',
    'cs': 'Czech',
    'sr': 'Serbian',
    'kn': 'Kannada',
    'gl': 'Galician',
    'vi': 'Vietnamese',
    'el': 'Greek',
    'bn': 'Bengali',
    'ml': 'Malayalam',
    'ro': 'Romanian',
    'pa': 'Punjabi',
    'fa': 'Persian',
    'ka': 'Georgian',
    'dz': 'Dzongkha',
    'eu': 'Basque',
    'km': 'Khmer',
    'hu': 'Hungarian',
    'la': 'Latin',
    'lv': 'Latvian',
    'nb': 'Norwegian Bokmål',
    'ur': 'Urdu',
    'kk': 'Kazakh'
}

In [428]:
movies['Language'] = movies['original_language'].map(language_mapping)

#### Drop the useless columns

In [429]:
movies.drop(columns=['original_language','overview'],inplace=True)

#### Date format for dates

In [430]:
movies['release_date'] = pd.to_datetime(movies['release_date'])

/var/folders/5q/f280wjd15s56_y9s5ntbx3rw0000gn/T/ipykernel_98386/1211608248.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  movies['release_date'] = pd.to_datetime(movies['release_date'])


#### Rename columns

In [431]:
movies.columns

Index(['title', 'release_date', 'popularity', 'vote_average', 'vote_count',
       'Language'],
      dtype='object')

In [432]:
movies.columns = ['Title', 'Release Date', 'Reach', 'Vote', 'Number of Votes', 'Language']

### Korean Movies

In [433]:
k_movies.dtypes

Name                  object
Year of release        int64
Aired Date            object
Aired On              object
Number of Episode      int64
Network               object
Duration              object
Content Rating        object
Synopsis              object
Cast                  object
Genre                 object
Tags                  object
Rank                  object
Rating               float64
dtype: object

#### Convert Broadcast Date to get a Start Date and an End Date

In [434]:
# Extract the dates while managing errors
def extract_dates(date_range):
    if pd.isna(date_range) or ' - ' not in date_range:
        return pd.Series([pd.NaT, pd.NaT], index=['start_date', 'end_date'])
    
    try:
        start_date_str, end_date_str = date_range.split(' - ')
        start_date = datetime.strptime(start_date_str, '%b %d, %Y')
        end_date = datetime.strptime(end_date_str, '%b %d, %Y')
        return pd.Series([start_date, end_date], index=['start_date', 'end_date'])
    except ValueError:
        return pd.Series([pd.NaT, pd.NaT], index=['start_date', 'end_date'])

# Extract dates
k_movies[['Start Date', 'End Date']] = k_movies['Aired Date'].apply(extract_dates)

#### Fill the NaT values in these 2 new columns

In [435]:
def fill_dates(row):
    if pd.isna(row['Start Date']) and pd.isna(row['End Date']):
        return pd.Series([row['Aired Date'], row['Aired Date']], index=['Start Date', 'End Date'])
    return pd.Series([row['Start Date'], row['End Date']], index=['Start Date', 'End Date'])

In [436]:
k_movies[['Start Date', 'End Date']] = k_movies.apply(fill_dates, axis=1) #Apply to the 2 columns created

#### Date format for Start Date and End Date

In [437]:
k_movies['Start Date'] = pd.to_datetime(k_movies['Start Date'],format='%b %d, %Y')

In [438]:
k_movies['End Date'] = pd.to_datetime(k_movies['End Date'],format='%b %d, %Y')

#### Convert the durations into minutes

In [439]:
# Convert durations into minutees
def convert_duration_to_minutes(duration):
    # Variables
    hours = 0
    minutes = 0

    # Find hours
    hours_match = re.search(r'(\d+)\s*hr\.?', duration)
    if hours_match:
        hours = int(hours_match.group(1))
    
    # Find minutes
    minutes_match = re.search(r'(\d+)\s*min\.?', duration)
    if minutes_match:
        minutes = int(minutes_match.group(1))
    
    # Convert into total minutes
    total_minutes = hours * 60 + minutes
    return total_minutes

# Apply the function
k_movies['Duration'] = k_movies['Duration'].apply(convert_duration_to_minutes)

#### Keep only the +Age in the Content Rating column

In [440]:
def extract_rating(rating):
    # Check if it is a string
    if isinstance(rating, str):
        match = re.match(r'(\d+\+)', rating)
        return match.group(1) if match else None
    return None 

# Apply the function to the column Content Rating
k_movies['Content Rating'] = k_movies['Content Rating'].apply(extract_rating)

#### Convert ranks into Integers

In [441]:
k_movies['Rank'] = k_movies['Rank'].str.strip('#').astype('int64')

#### Split Network in 2 columns to manage the programs broadcast on several networks

In [442]:
k_movies[['Network 1', 'Network 2']] = k_movies['Network'].str.split(', ', expand=True)

In [443]:
k_movies['Network 1'] = k_movies['Network 1'].str.strip()

In [444]:
k_movies['Network 2'] = k_movies['Network 2'].str.strip()

#### Split Genre in 2 columns to keep only the 2 main ones

In [445]:
# Separate data using the comma separator
split_categories = k_movies['Genre'].str.split(', ', expand=True)

# Keep only the 2 main genres
k_movies[['Genre 1', 'Genre 2']] = split_categories.iloc[:, :2]

In [446]:
# Separate data using the comma separator
split_cast = k_movies['Cast'].str.split(', ', expand=True)

# Keep only the 2 main actors
k_movies[['Main Actor', 'Second Role']] = split_cast.iloc[:, :2]

#### Remove useless columns

In [447]:
k_movies.columns

Index(['Name', 'Year of release', 'Aired Date', 'Aired On',
       'Number of Episode', 'Network', 'Duration', 'Content Rating',
       'Synopsis', 'Cast', 'Genre', 'Tags', 'Rank', 'Rating', 'Start Date',
       'End Date', 'Network 1', 'Network 2', 'Genre 1', 'Genre 2',
       'Main Actor', 'Second Role'],
      dtype='object')

In [448]:
k_movies.drop(columns=['Aired Date','Network','Synopsis','Cast','Genre'],inplace=True)

In [449]:
k_movies

,Name,Year of release,Aired On,Number of Episode,Duration,Content Rating,Tags,Rank,Rating,Start Date,End Date,Network 1,Network 2,Genre 1,Genre 2,Main Actor,Second Role
0,Move to Heaven,2021,Friday,10,52,18+,"Autism, Uncle-Nephew Relationship, Death, Sava...",1,9.2,2021-05-14,2021-05-14,Netflix,None,Life,Drama,Lee Je Hoon,Tang Jun Sang
1,Hospital Playlist,2020,Thursday,12,90,15+,"Strong Friendship, Multiple Mains, Best Friend...",2,9.1,2020-03-12,2020-05-28,Netflix,tvN,Friendship,Romance,Jo Jung Suk,Yoo Yeon Seok
2,Flower of Evil,2020,"Wednesday, Thursday",16,70,15+,"Married Couple, Deception, Suspense, Family Se...",3,9.1,2020-07-29,2020-09-23,tvN,None,Thriller,Romance,Lee Joon Gi,Moon Chae Won
3,Hospital Playlist 2,2021,Thursday,12,100,15+,"Workplace, Strong Friendship, Best Friends, Mu...",4,9.1,2021-06-17,2021-09-16,Netflix,tvN,Friendship,Romance,Jo Jung Suk,Yoo Yeon Seok
4,My Mister,2018,"Wednesday, Thursday",16,77,15+,"Age Gap, Nice Male Lead, Strong Female Lead, H...",5,9.1,2018-03-21,2018-05-17,tvN,None,Psychological,Life,Lee Sun Kyun,IU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Vagabond,2019,"Friday, Saturday",16,70,15+,"Plane Crash, Revenge, National Intelligence Se...",96,8.5,2019-09-20,2019-11-23,Netflix,SBS,Action,Thriller,Lee Seung Gi,Bae Suzy
96,Doctor John,2019,"Friday, Saturday",32,35,15+,"Smart Male Lead, Incurable Disease, Eccentric ...",97,8.5,2019-07-19,2019-09-07,SBS,None,Mystery,Romance,Ji Sung,Lee Se Young
97,When the Camellia Blooms,2019,"Wednesday, Thursday",40,35,15+,"Single Mother, Mystery, Motherhood, Serial Kil...",98,8.5,2019-09-18,2019-11-21,KBS2,Netflix,Thriller,Comedy,Gong Hyo Jin,Kang Ha Neul
98,Because This Is My First Life,2017,"Monday, Tuesday",16,70,15+,"Contract Relationship, Marriage Of Convenience...",99,8.5,2017-10-09,2017-11-28,tvN,None,Comedy,Romance,Jung So Min,Lee Min Ki


## Export

### ATP

In [452]:
atp.to_csv('Outputs/atp.csv')

### World Cup

In [453]:
wc22.to_csv('Outputs/wc22.csv')

### Sports Events